In [ ]:
%%!
pip install --trusted-host pypi.org ipython-sql

In [2]:
%load_ext sql
%sql sqlite://
import sqlite3
import pandas as pd

In [4]:
df1 = pd.read_csv("/Users/apple/Desktop/takehomechallenges/sql/query_five/query_five_transactions.csv")
df2=pd.read_csv("/Users/apple/Desktop/takehomechallenges/sql/query_five/query_five_users.csv")

In [5]:
conn = sqlite3.connect("query_five.db")

In [6]:
df1.to_sql("transactions",conn)
df2.to_sql("users",conn)

In [106]:
sql_string ='''
SELECT AVG(amount) AS average,  
       AVG(CASE WHEN row_num BETWEEN row_num_desc-1 and row_num_desc+1 THEN amount ELSE NULL END ) AS median  
from(
select users.user_id, transactions.transaction_amount as amount,
row_number()over(order by transaction_amount) as row_num,
row_number()over(order by transaction_amount desc) as row_num_desc
from users
inner join transactions
on users.sign_up_date=date(transactions.transaction_date) and users.user_id=transactions.user_id
order by transaction_amount)

'''

In [107]:
df = pd.read_sql(sql_string,conn)
df

,average,median
0,49.246257,None


In [112]:
sql_string ='''
SELECT AVG(transaction_amount) AS average,  
       AVG(CASE WHEN row_num_asc BETWEEN row_num_desc-1 and row_num_desc+1 THEN transaction_amount ELSE NULL END ) AS median  
FROM  
    (SELECT transaction_amount,  
             ROW_NUMBER() OVER(ORDER BY transaction_amount) row_num_asc,  
             COUNT(*) OVER() - ROW_NUMBER() OVER(ORDER BY transaction_amount) + 1 AS row_num_desc -- need row number for median. there are many other ways to do this  
     FROM users a  
     JOIN (SELECT *, 
           date(transaction_date) AS date_only 
           FROM transactions
           ) b  
      ON a.user_id = b.user_id AND a.sign_up_date = b.date_only  
) tmp;
'''

In [113]:
df = pd.read_sql(sql_string,conn)
df

,average,median
0,49.246257,49.0
